In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container {width : 99% !important;}</style>"))
display(HTML("<style>.output_result {width : 99% !important;}</style>"))
display(HTML("<style>.jp-Notebook {--jp-notebook-max-width: 99%;}/style>"))

In [2]:
import pyspark
sc = pyspark.SparkContext().getOrCreate()
spark = pyspark.sql.SparkSession(sc)
sc
spark

In [3]:
import pandas as pd
import numpy as np

# Recommendations Are Everywhere

## Why learn how to build recommendation engines?
1. Why learn how to build recommendation engines?

Hi. Welcome to this course on building recommendation engines using Alternating Least Squares or "ALS" in PySpark.

2. What recommendations look like
00:09 - 00:34
You're probably already familiar with the output of these types of recommendation engines where a website tells you something along the lines of, "If you like that, then you'll probably like this." You've likely seen these types of recommendations on your favorite retail or media streaming websites. These recommendations are generated through different types of data that you as a user or customer provide either directly or indirectly.

3. Learning about you

When you purchase something online, or watch a movie, or even read an article, you are often given a chance to rate that item on a scale of 1 to 5 stars, a thumbs up or thumbs down, or some other type of rating scale. Based on your feedback from these types of rating systems, companies can learn a lot about your preferences, and offer you recommendations based on preferences of users that are similar to you.

4. How recommendation engines work

For example, if your movie streaming service sees that you liked Dark Knight and Iron Man, and did not like Tangled, and it also sees

5. How recommendation engines work

other users that also liked Dark Knight and Iron Man and also did not like Tangled, the ALS algorithm would see that you and these other users have

6. How recommendation engines work

similar tastes. It would then look at the movies that you have not yet seen, and see which ones are the

7. How recommendation engines work

highest rated among those similar users, and offer them as

8. How recommendation engines work

recommendations to you. This is why websites will often say things like, "Because you liked that movie, we think you'll like this movie." Or "Users like you also watched this movie."

9. The Power of Recommendation Engines

These types of rating systems are extremely powerful. In fact, an article published by McKinsey & Company in October of 2013 stated that 35% of what customers buy on Amazon and 75% of what they watch on Netflix come from product recommendations based on algorithms such as the one you are going to be learning in this course. That's a powerful use of data, and with this course, you will learn how to do this. In addition to this, there are alternate uses for recommendation algorithms that can be extremely useful for purposes as broad as feature space reduction, image compression, mathematical user and product grouping, latent feature discovery and you're going to learn some of these in this course.

10. Prerequisites

This tutorial is intended for those that have experience with Spark and Python, and understand the fundamentals of machine learning. If needed, some good introductory resources are DataCamp's Introduction to PySpark course, their Intermediate Python for Data Science course, and their Supervised Machine Learning with Python's SciKitLearn course.

11. Let's practice!

Let's jump in.

### See the power of a recommendation engine
Taylor and Jane both like watching movies. Taylor only likes dramas, comedies, and romances. Jane likes only action, adventure, and otherwise exciting films. One of the greatest benefits of ALS-based recommendation engines is that they can identify movies or items that users will like, even if they themselves think that they might not like them. Take a look at the movie ratings that Taylor and Jane have provided below. It would stand to reason that their different preferences would generate different recommendations.

This course touches on a lot of concepts you may have forgotten, so if you ever need a quick refresher, download the [PySpark Cheat Sheet](https://datacamp-community-prod.s3.amazonaws.com/65076e3c-9df1-40d5-a0c2-36294d9a3ca9) and keep it handy!

**Instructions**

- Take a look at `TJ_ratings` using the `.show()` method and any other methods you prefer to see how each of them rated the various movies they've seen.
- Input user names into the `get_ALS_recs()` function provided to see what movies ALS recommends for Jane and Taylor based on the ratings provided. Do the ratings make sense to you?

#### To get code for the function, used:
`import inspect`

`lines = inspect.getsource(get_ALS_recs)`

`print(lines)`

In [4]:
TJ_pandas = pd.DataFrame(columns = ['user_name' ,'movie_name' , 'rating'] , data = np.array([ ['Taylor'       ,          'Twilight'    , 4.9]
,     ['Taylor'   ,    'A Walk to Remember' ,    4.5]
,     ['Taylor'   ,          'The Notebook'   ,  5.0]
,     ['Taylor'  ,'Raiders of the Lost Ark'   ,  1.2]
,     ['Taylor'  ,         'The Terminator'  ,   1.0]
,     ['Taylor'   ,        'Mrs. Doubtfire' ,    1.0]
,     [  'Jane'  ,               'Iron Man'  ,   4.8]
,      [ 'Jane' , 'Raiders of the Lost Ark'  ,   4.9]
,      [ 'Jane' ,          'The Terminator'  ,   4.6]
,    [   'Jane'  ,              'Anchorman'   ,  1.2]
,   [   'Jane'   ,          'Pretty Woman'   ,  1.0]
,   [   'Jane'   ,             'Toy Story'  ,   1.2]]))

TJ_ratings = spark.createDataFrame(TJ_pandas)
TJ_ALS_recs = spark.createDataFrame(pd.DataFrame(columns = ['userId','pred_rating' , 'title', 'genres'], data = np.array([  [ 'Taylor'   ,      3.89  , 'Seven Pounds (2008)',           'Drama']
,   ['Taylor'       ,  3.61 ,     'Cure, The (1995) ',          'Drama']
,   ['Taylor'     ,    3.55 , 'Kiss Me, Guido (1997 ',         'Comedy']
,   ['Taylor'    ,     3.29 , 'You\'ve Got Mail (199 ', 'Comedy|Romance']
,   ['Taylor'    ,     3.27 , '10 Things I Hate Abo' , 'Comedy|Romance']
,   ['Taylor'    ,     3.26 , 'Corrina, Corrina (19' , 'Comedy|Drama|R']
,   [  'Jane'    ,     4.96 ,          'Fear (1996)' ,       'Thriller']
,   [  'Jane'    ,     4.85 , 'Lord of the Rings: T'  ,'Adventure|Fant']
,   [  'Jane'    ,     4.70 , 'Lord of the Rings: T ', 'Adventure|Fant']
,   [  'Jane'    ,     4.55,  'No Holds Barred (198 ',         'Action']
,  [  'Jane'    ,     4.54 , 'Lord of the Rings: T' , 'Action|Adventu']
, [   'Jane'    ,     4.30 , 'Band of Brothers (20' , 'Action|Drama|W']
,  [  'Jane'    ,     4.26  , 'Transformers (2007)' , 'Action|Sci-Fi|']
])
))


def get_ALS_recs(list_of_user_names, recs = TJ_ALS_recs):
    """ 
    Returns recommendations generated by ALS algorithm
    
    Parameters:
      - list_of_user_names: List or array of user names
      - recs: ALS-generated recommendations
      
    Returns: Pyspark dataframe of recommendations for user names submitted in list_of_usre_names.
    """
    if len(list_of_user_names) == 0:
        print ("None")
    elif len(list_of_user_names) == 1:
        print (recs[recs.userId == list_of_user_names[0]].sort_values(by=['user_name','pred_rating'], ascending = [0,0]))
    elif len(list_of_user_names) == 2:
        print (recs)
    return None

In [5]:
# View TJ_ratings
TJ_ratings.show()

# Generate recommendations for users
get_ALS_recs(["Taylor","Jane"]) 

+---------+--------------------+------+
|user_name|          movie_name|rating|
+---------+--------------------+------+
|   Taylor|            Twilight|   4.9|
|   Taylor|  A Walk to Remember|   4.5|
|   Taylor|        The Notebook|   5.0|
|   Taylor|Raiders of the Lo...|   1.2|
|   Taylor|      The Terminator|   1.0|
|   Taylor|      Mrs. Doubtfire|   1.0|
|     Jane|            Iron Man|   4.8|
|     Jane|Raiders of the Lo...|   4.9|
|     Jane|      The Terminator|   4.6|
|     Jane|           Anchorman|   1.2|
|     Jane|        Pretty Woman|   1.0|
|     Jane|           Toy Story|   1.2|
+---------+--------------------+------+

DataFrame[userId: string, pred_rating: string, title: string, genres: string]


### Power of recommendation engines
What is a reason for learning to build recommendation engines?

Answer the question

- Sales always go up with recommendations.

- **Show users items/products relevant to them that they may not know are available.**

- Customers always take recommendations.

- Because other successful companies do it.

## Recommendation engine types and data types

1. Recommendation engine types and data types

In the world of recommendation engines, there are two basic types:

2. Two types of recommendation engines:

Collaborative-filtering engines and content-based filtering engines. Both aim to offer meaningful recommendations, but they do so in slightly different ways. Content-based filtering, as the name suggests, tries to understand the content, or {{1}}features of the items, and makes recommendations based on your preferences for those specific features. For example, a movie streaming service might go to great lengths to add descriptive tags to their movies such as the genre, whether it's animated or not, the language spoken in the movie, the decade it was filmed, and which actors were in it, etc. So when a user like you

3. Two types of recommendation engines

gives 5 stars to a really dramatic, Portuguese movie with specific actors from a specific decade, they can infer that you like movies like this and will also like

4. Two types of recommendation engines

other dramatic movies in Portuguese with those same actors,

5. Two types of recommendation engines

and recommend those movies to you. Collaborative filtering is a little bit different.

6. Two types of recommendation engines

As explained in the previous video, collaborative filtering is based on user similarity. However, unlike content-based filtering, manually-created tags are not necessary. The features and groupings are created mathematically from patterns in the ratings provided by users. When you provide ratings for a product or item, whether it be a thumbs up or thumbs down, or even if you just watch a video without even giving it a rating, you are providing meaningful insight about your preferences. From this behavior, the ALS algorithm can mathematically group you with similar users, predict your behavior, and help you have a more effective customer experience. While ALS can have content-based applications, this course will focus on it's application to collaborative filtering, but many of the principles of collaborative filtering can be applied to content-based applications.

7. Two types of ratings

Now let's talk about ratings. In the realm of recommendation engines, there are two main types of ratings:

8. Two types of ratings

Explicit ratings

9. Two types of ratings

and implicit ratings Explicit ratings are pretty straightforward. Examples of these are when you input

10. Two types of ratings

a number of stars or something like a thumbs up or thumbs down. These are explicit ratings because users explicitly state how much they like or dislike something. Implicit ratings are a little bit different. They are based on the passive tracking of your behavior, like the number of movies you've seen in different genres. Fundamentally, implicit ratings are generated from the frequency of your actions. For example, if you watch 30 movies, and of those 30 movies,

11. Two types of ratings

22 are action movies, and only 1 is a comedy, the low number of comedy views will be converted into

12. Two types of ratings

low confidence that you like comedies, and the high number of action movie views will be converted into

13. Two types of ratings

a high confidence that you like action movies. These probabilities are then used as ratings. The logic behind this is, in essence, the more you carry out a behavior, the higher the the likelihood that you like it, and thus a higher rating. Additionally, in some cases you may not have access to behavior counts like this. A simpler form of ratings that still works with the ALS algorithm is the use of simple binary ratings. Rather than having a count of user actions, binary ratings just show whether a user has done something, like watched a comedy, represented by a 1 or not watched a comedy, represented by a 0. These types of ratings aren't nearly as rich, but they still can provide meaningful insight and still work perfectly fine with the ALS algorithm.

14. Let's practice!

Now, let's look at some actual data.

### Collaborative vs content-based filtering
Below are statements that are often used when providing recommendations. Select the one that DOES NOT indicate collaborative filtering.

Answer the question

- **"Because you liked that product, we think you'll like this product."**

- "Users that bought that also bought this."

- "Other people like you also liked this movie."

- "80% of your friends liked this movie, we think you'll like it too."

- "Here are top choices from similar users."

### Collaborative vs content based filtering part II
Look at the df dataframe using the `.show()` method and/or the `.columns` method, and determine whether it is best suited for "collaborative filtering", "content-based filtering", or "both".

- collaborative filtering
- content-based filtering
- **both**

### Implicit vs explicit data
Recall the differences between implicit and explicit ratings. Take a look at the `df1` dataframe to understand whether the data includes implicit or explicit ratings data.

**Instructions**

- Use the `.columns` and `.show()` methods to get an idea of the data provided, and see if the data includes implicit or explicit ratings.
- Type "implicit" or "explicit" based on whether you think this data contains "implicit" ratings or "explicit" ratings. Name your response answer.

In [6]:
df1 = spark.read.csv("markus_ratings.csv", inferSchema = True, header= True)

In [7]:
df1.show()

+--------------------+------------------+---------+
|         Movie_Title|             Genre|Num_Views|
+--------------------+------------------+---------+
|        Finding Nemo|Animated Childrens|       12|
|           Toy Story|Animated Childrens|        6|
|            Iron Man|            Action|        1|
|     Captain America|            Action|        1|
|     The Incredibles|Animated Childrens|        9|
|              Frozen|Animated Childrens|       22|
|The Shawshank Red...|             Drama|        2|
|  Rabbit Proof Fence|             Drama|        2|
|Searching for Sug...|       Documentary|        3|
|              Powder|             Drama|        1|
|        The Fugitive|            Action|        2|
+--------------------+------------------+---------+



In [8]:
# Type "implicit" or "explicit"
answer = "implicit"

### Ratings data types
Markus watches a lot of movies, including documentaries, superhero movies, classics, and dramas. Drawing on your previous experience with Spark, use the `markus_ratings` dataframe, which contains data on the number of times Markus has seen movies in various genres, and think about whether these are implicit or explicit ratings. Use the `groupBy()` method to determine which genre has the highest rating, which could likely influence what recommendations ALS would generate for Markus.

**Instructions**

- Use the `groupBy()` method to group the `markus_ratings` dataframe by `"Genre"`.
- Apply the `.sum()` method to get the total number of movies watched for each genre.
- Be sure to add the `.show()` method at the end to view the counts.

In [9]:
markus_ratings = df1

In [10]:
# Group the data by "Genre"
markus_ratings.groupBy("Genre").sum().show()

+------------------+--------------+
|             Genre|sum(Num_Views)|
+------------------+--------------+
|             Drama|             5|
|       Documentary|             3|
|            Action|             4|
|Animated Childrens|            49|
+------------------+--------------+



## Uses for recommendation engines
1. Uses for recommendation engines

So far, we've only considered recommendations as the use case for the ALS algorithm, but there are other applications that are also useful. These include latent feature discovery, item grouping, dimensionality reduction and image compression. In this course we'll only talk about some of these. First let's talk about latent features. As mentioned earlier, people will go to great lengths to effectively categorize items. But some products span various categories making them difficult to organize. Movies are often like this. Horror movies can be comedies. Dramas can be satires. Documentaries can be romances or even mysteries. Because of this, they can sometimes be difficult to market. If we had a better understanding of how consumers categorize movies based on their experience watching them, we could add more power to marketing strategies. ALS can help with this.

2. Basic factorization of ALS

When we have a matrix that contains users and movie ratings, ALS

3. Basic factorization of ALS (cont.)

will factor that matrix into two matrices, one containing

4. User matrix

user information and the other containing

5. Product matrix

product information, or in this case, movie information. Each matrix takes the respective

6. Factor matrix axes

labeled axis from the original matrix, and is given another axis that is unlabeled. The unlabeled axes contain what’s called

7. Latent feature axes

latent features. The number of latent features is referred to as the "rank" of these matrices. In this case, the rank chosen is 3. You, as a data scientist get to choose how many of these ALS will create. These latent features represent groups that are created from patterns in the original ratings matrix and the values in these columns represent how much each item falls into these groups. For example,

8. Horror vs drama

in the original ratings matrix, there might be a lot of people who like horror movies and don’t like dramas. They would rate

9. Horror vs Drama Part II

horror movies high

10. Horror vs Drama Part III

and dramas low. Likewise other people might like dramas and not like horror movies, and would rate

11. Horror vs Drama Part IV

dramas high

12. Horror vs Romance Part V

and horror films low. ALS can see this and determine that these are different types of movies.

13. Horror vs Drama Part VI

And if we were to look at the movie factor matrix, we would likely see that in one of the latent feature rows the dramas would score high, and the horror movies would score low, while in another latent feature column we might see the

14. Horror vs Drama Part VII

opposite. Knowing a little about these movies, we could determine that those latent features reflect those two genres. This allows us to mathematically see how users experience these movies and to what degree users feel each movie falls into each respective category. This concept goes a bit deeper though. For example, if we were to look at a

15. Uncovering Features

movie matrix, we might see in one latent-feature column,

16. Uncovering Features Part II

that several movies have scored very high, but they

17. Uncovering Features Part III

don’t seem to have anything in common. If they are all popular movies,

18. Uncovering Features Part IV

we might want to research what’s going on here to see if there's a business opportunity. Digging deeper,

19. Shakespeare Adaptations

we find that these movies are all adaptations of Shakespeare plays, and that there seems to be a strong customer group that likes these types of movies. Now that we know this, we can use this information to inform how we choose what movies to make and hopefully give our customers more of what they want. It’s worth reiterating that in the original data set there was no column anywhere called “Shakespeare Adaptations”. It’s also worth noting that many or all of these customers may not even know that this is something that draws them to these movies. This is the type of powerful information that ALS can help us uncover.

20. Let's practice!

Now lets try and actually use this.

### Alternate uses of recommendation engines.
Select the best definition of "latent features".

Answer the question

- Features or tags that have manually been attached to items that categorize those items.

- **Features that are contained in data, but aren't directly observable.**

- Features that show up "later" in the machine learning process.

- Features that are added by human beings.

### Confirm understanding of latent features
Matrix `P` is provided here. Its columns represent movies and its rows represent several latent features. Use your understanding of Spark commands to view matrix `P` and see if you can determine what some of the latent features might represent. After examining the matrix, look at the dataframe `Pi`, which contains a rough approximation of what these latent features could represent. See if you weren't far off.

**Instructions**

- Examine matrix `P` using the `.show()` method.
- Examine matrix `Pi` using the `.show()` method.

In [11]:
P = spark.read.csv("MatrixP.csv", inferSchema = True, header = True)
Pi = spark.read.csv("MatrixPi.csv", inferSchema = True, header= True)

In [12]:
# Examine matrix P using the .show() method
P.show()

# Examine matrix Pi using the .show() method
Pi.show()

+--------+------------+--------+---------+------------+------+----------+
|Iron Man|Finding Nemo|Avengers|Toy Story|Forrest Gump|Wall-E|Green Mile|
+--------+------------+--------+---------+------------+------+----------+
|     0.2|         2.4|     0.1|      2.4|         0.0|   2.5|       0.0|
|     1.5|         1.4|     1.4|      1.3|         1.8|   1.8|       2.5|
|     2.5|         1.1|     2.4|      0.9|         0.2|   0.9|      0.09|
|     1.9|         2.0|     1.5|      2.2|         1.2|   0.3|      0.01|
|     0.0|         0.0|     0.0|      2.3|         2.2|   0.0|       2.5|
+--------+------------+--------+---------+------------+------+----------+

+---------+--------+------------+--------+---------+------------+------+----------+
| Lat Feat|Iron Man|Finding Nemo|Avengers|Toy Story|Forrest Gump|Wall-E|Green Mile|
+---------+--------+------------+--------+---------+------------+------+----------+
| Animated|     0.2|         2.4|     0.1|      2.4|         0.0|   2.5|       0.

# How does ALS work

## Overview of matrix multiplication
1. Matrix Multiplication

As you've probably realized, matrix operations are fundamental to the ALS algorithm. We're going to review matrix multiplication and matrix factorization here. Let's start with multiplication.

2. Matrix Multiplication

Here we have two square matrices. In order to multiply them together, we make specific pairs of the values from the two matrices, and add the products of those pairs. We start at the top left-hand corner of each matrix, and create pairs moving to the right on the first matrix, and moving down on the second matrix one at a time. Each pair is multiplied, and the products from all pairs are added together. The final sum will make up one number of the resulting matrix. That's a lot to digest, so let's walk through an example. Starting at the top

3. Matrix Multiplication

left number of each matrix we have a pair of numbers, 1 and 9. We will multiply those numbers together. Then moving to the right on the first matrix, and down on the second matrix we have

4. Matrix Multiplication

2 and 6, then moving

5. Matrix Multiplication

right again on the first matrix and down again on the second matrix we have 3 and 3. We have completed the first set of pairs, so let's add their products together. 1 times 9, plus 2 times 6 plus 3 times 3 is

6. Matrix Multiplication

9 plus 12 plus 9, which gives us

7. Matrix Multiplication

30. 30 is the first number in our final matrix. From here we stay on the first row of the first matrix, but move on to the second column of the

8. Matrix Multiplication

second matrix. These pairs give us 1 and 8,

9. Matrix Multiplication

2 and 5,

10. Matrix Multiplication

and 3 and 2. 1 times 8 plus 2 times 5 plus 3 times 2 is equal to 8 plus 10 plus 6, which is

11. Matrix Multiplication

24. Moving to the next set of pairs, we multiply

12. Matrix Multiplication

1 and 7, 2 and 4, and 3 and 1. Their products are 7, 8 and 3 which makes

13. Matrix Multiplication

18. Once we've multiplied the first row of the first matrix by all columns of the second matrix, we then go through the same process for

14. Matrix Multiplication

the second row of the first matrix with all the columns of the second matrix,

15. Matrix Multiplication

and so on

16. Matrix Multiplication

until all rows of the

17. Matrix Multiplication

first matrix have been multiplied

18. Matrix Multiplication

by all columns of the second matrix. In this example, we multiplied two square matrices of the same dimensions. In reality, you can multiply any two matrices as long as the

19. Matrix Multiplication

number of columns of the first matrix matches the number of rows of the second matrix,

20. Matrix Multiplication

If they don't, then some values in one of the matrixces won't be paired, and multiplication can't be completed.

21. Let's practice!

Let's look at some examples, and practice matrix multiplication.

### Matrix multiplication
To understand matrix multiplication more directly, let's do some matrix operations manually.

**Instructions**

- Matrices `a` and `b` are Pandas dataframes. Use the `.head()` method on each of them to view them.
- Work out the product of these two matrices on your own.
- Enter the values of the product of the `a` and `b` matrices into the product array, created using `np.array()`.
- Use the validation on the last line of code to evaluate your estimate. The `.dot()` method multiplies two matrices together.

In [13]:
a = pd.DataFrame(columns = [0,1], index= ['one', 'two'], data = np.array([[2,2], [3,3]]))
b = pd.DataFrame(columns = [0,1], index= ['one', 'two'], data = np.array([[1,2], [4,4]]))
a.head()


,0,1
one,2,2
two,3,3


In [14]:
b.head()

,0,1
one,1,2
two,4,4


In [15]:
# Use the .head() method to view the contents of matrices a and b
print("Matrix A: ")
print (a.head())

print("Matrix B: ")
print (b.head())

# Complete the matrix with the product of matrices a and b
product = np.array([[10,12], [15,18]])

# Run this validation to see how your estimate performs
product == np.dot(a,b)

Matrix A: 
     0  1
one  2  2
two  3  3
Matrix B: 
     0  1
one  1  2
two  4  4


array([[ True,  True],
       [ True,  True]])

### Matrix multiplication part II
Let's put your matrix multiplication skills to the test.

**Instructions**

- Use the `.shape` attribute to understand the dimensions of matrices `C` and `D`, and determine whether these two matrices can be multiplied together or not.
- If they can be multiplied, use the `np.matmul()` method to multiply them. If not, set `C_times_D` to `None`.

In [16]:
C = pd.DataFrame(data = np.array([[3 , 4 , 5 , 1,  2], [2 , 5  ,7  ,6  ,8], [1,  9  ,0  ,7  ,6], [2 , 2  ,3 , 3 ,1]])) 
D = pd.DataFrame(data = np.array([[1, 2], [3, 3],[9,8]]))
C.head()

,0,1,2,3,4
0,3,4,5,1,2
1,2,5,7,6,8
2,1,9,0,7,6
3,2,2,3,3,1


In [17]:
D.head()

,0,1
0,1,2
1,3,3
2,9,8


In [18]:
# Print the dimensions of C
print(C.shape)

# Print the dimensions of D
print(D.shape)

# Can C and D be multiplied together?
C_times_D = None

(4, 5)
(3, 2)


That's right. The number of columns in `C` is different than the number of rows in `D`. `C` and `D` cannot be multiplied.

## Overview of matrix factorization
1. Overview of matrix factorization

Matrix factorization, or matrix decomposition, is essentially the opposite of matrix multiplication. Rather than multiplying two matrices together to get one new matrix, matrix factorization

2. Matrix Factorization

splits a matrix into two or more matrices which, when multiplied back together,

3. Matrix Factorization

produce an approximation of the original matrix. There are several different mathematical approaches for this, each of which has a different application. We aren't going to go into any of that here, we are simply going to review the factorization that ALS performs. Used in the context of collaborative filtering, ALS uses a factorization called non-negative matrix factorization. Because matrix factorization generally returns only approximations of the original matrix, in some cases, they can return negative values in the factor matrices, even when attempting to predict positive values. When predicting what rating a user will give to an item, negative values don't really make sense. Neither do they make sense in the context of latent features. For this reason, the version of ALS that we will use will require that the factorization return only positive values. Let's look at some sample factorizations.

4. Matrix Factorization

Here is a sample matrix of possible item ratings. There are 5 rows and 5 columns. And here

5. Matrix Factorization

is one factorization of that matrix called the LU factorization. Notice that the factor matrices are the same dimensions or rank as the original matrix. Also notice that some of the values in this factorization are negative. Using this type of factorization could result in negative predictions that wouldn't make sense in our context.

6. Matrix Factorization

Here is another factorization. In this case, all the values are positive meaning the resulting product of these factor matrices is guaranteed to be positive. This is closer to what we need for our purposes. Notice here that the dimensions of the factor matrices are such that the first factor matrix has the same number of rows as the original matrix, but a different number of columns. Also, the second factor matrix has the same number of columns as the original matrix, but a different number of rows. The dimensions of the factor matrices that don't match the original matrix are called the "rank" or number of latent features. In this case, we have chosen the "rank" of the factor matrices to be 3. What that means is that the number of

7. Rank of Factor Matrices

latent features of the factor matrices is 3. Remember that as a data scientist, when doing these types of factorizations, you get to choose the rank, or number of latent features the factor matrices will have.

8. Filling in the Blanks

Now look at this matrix. Not all the cells have numbers in them. Despite this, we can still

9. Filling in the Blanks

factor the values in the matrix. Also notice that because there is at least

10. Filling In the Blanks

one value in

11. Filling In the Blanks

every row and at

12. Filling In the Blanks

least one value in every column

13. Filling In the Blanks

that each of the factor matrices are totally full. Because of this, factoring a sparse matrix into two factor matrices gives us the means to not only approximate the original values that existed in the matrix to begin with, but to also

14. Filling In the Blanks

provide predictions for the cells that were originally blank. And because the factorization is based on the values that existed previously, the blank cells are filled in based on those already-existing patterns. So when we do this with user ratings, the blanks are filled in with values that reflect the individual user behavior and the behavior of users similar to them. This is why this method is called collaborative filtering.

15. Let's practice!

Let's look at some real-life examples of this.

### Matrix factorization
Matrix `G` is provided here as a Pandas dataframe. View it to understand what it looks like. Look at the possible factor matrices `H`, `I`, and `J` (also Pandas dataframes), and determine which two matrices will produce the matrix `G` when multiplied together.

**Instructions**

- Take a look at matrix `G` using the print command
- Take a look at the matrices `H`, `I`, and `J` and determine which pair of those matrices will produce `G` when multiplied together.
- Input your answer into the `np.matmul()` code provided.

In [19]:
G = pd.DataFrame(data = np.array([[6, 6], [3, 3]]))
H = pd.DataFrame(data = np.array([[2, 2], [1, 1]]))
I = pd.DataFrame(data = np.array([[3, 3], [3, 3]]))
J = pd.DataFrame(data = np.array([[1, 1], [2, 2]]))
G

,0,1
0,6,6
1,3,3


In [20]:
H

,0,1
0,2,2
1,1,1


In [21]:
I

,0,1
0,3,3
1,3,3


In [22]:
J

,0,1
0,1,1
1,2,2


In [23]:
# Take a look at Matrix G using the following print function
print("Matrix G:")
print(G)

# Take a look at the matrices H, I, and J and determine which pair of those matrices will produce G when multiplied together. 
print("Matrix H:")
print(H)
print("Matrix I:")
print(I)
print("Matrix J:")
print(J)

# Multiply the two matrices that are factors of the matrix G
prod = np.matmul(H, J)
print(G == prod)

Matrix G:
   0  1
0  6  6
1  3  3
Matrix H:
   0  1
0  2  2
1  1  1
Matrix I:
   0  1
0  3  3
1  3  3
Matrix J:
   0  1
0  1  1
1  2  2
      0     1
0  True  True
1  True  True


### Non-negative matrix factorization
It's possible for one matrix to have two equally close factorizations where one has all positive values and the other has some negative values.

The matrix `M` has been factored twice using two different factorizations. Take a look at each pair of factor matrices `L` and `U`, and `W` and `H` to see the differences. Then use their products to see that they produce essentially the same product.

**Instructions**

- Use `print()` to view the `L` and `U` matrices. Notice that some values in matrices `L` and `U` are negative.
- Use `print()` to view the `W` and `H` matrices. Notice that all values in these two matrices are positive.
- The `L` and `U` matrices and `W` and `H` matrices have been multiplied together to produce the `LU` and `WH` matrices respectively. Use `getRMSE`(`product_matrix`, `original_matrix`) to see how close `LU` is to `M` compared to how close `WH` is to `M`. Are they similar?

To get the source for getRMSE function I used:

`import inspect`

`lines = inspect.getsource(getRMSE)`

`print(lines)`

More details [here](https://stackoverflow.com/questions/427453/how-can-i-get-the-source-code-of-a-python-function)


In [24]:
def getRMSE(pred, actual):
    """
    Returns RMSE between predictions and actual observations
    
    Parameters: 
        predictions: pandas dataframe of value predictions
        actual values: pandas dataframe of actual values that predictions are trying to predict
        
    Returns: RMSE value in decimal format
    """
    RMSE =  (((pred - actual)**2).sum().sum()/(pred.shape[0]*pred.shape[1]))**.5
    return round(RMSE,3)

M = pd.DataFrame(data = np.array([ [1,  2 , 1,  2], [  0 , 0 , 0 , 0] , [  1 , 2 , 2 , 1], [  0 , 0 , 0 , 0] ]))

L = pd.DataFrame(data = np.array([[1.00 , 0.000000 , 0.000000 , 0],[  0.01, -0.421053 , 0.098316 , 1], [ 1.00  ,0.000000 , 1.000000 , 0], [0.10 , 1.000000 , 0.000000 , 0]]))
U = pd.DataFrame(data = np.array([   [ 1 , 2.00 , 1.000 , 2.000000],  [0 ,-0.19 ,-0.099 ,-0.198000],  [0 , 0.00 , 1.000 ,-1.000000],  [0 , 0.00,  0.000,  0.194947]]))

W = pd.DataFrame(data = np.array([   [ 2.61 , 0.24 , 0.00 , 0.12],  [0.00,  0.05,  0.02,  0.17], [ 1.97 , 0.00 , 0.58 , 0.83], [ 0.05 , 0.00 , 0.00 , 0.00]]))
H = pd.DataFrame(data= np.array([    [0.38 , 0.65,  0.34 , 0.41],  [0.00,  1.20 , 0.15  ,3.72],  [0.42 , 1.09 , 1.38 , 0.07],  [0.00 , 0.11 , 0.65 , 0.17]]))

LU = np.matmul(L,U)
WH = np.matmul(W,H)

In [25]:
# View the L, U, W, and H matrices.
print("Matrices L and U:") 
print(L)
print(U)

print("Matrices W and H:")
print(W)
print(H)

# Calculate RMSE between LU and M
print("RMSE of LU: ", getRMSE(LU, M))

# Calculate RMSE between WH and M
print("RMSE of WH: ", getRMSE(WH, M))

Matrices L and U:
      0         1         2    3
0  1.00  0.000000  0.000000  0.0
1  0.01 -0.421053  0.098316  1.0
2  1.00  0.000000  1.000000  0.0
3  0.10  1.000000  0.000000  0.0
     0     1      2         3
0  1.0  2.00  1.000  2.000000
1  0.0 -0.19 -0.099 -0.198000
2  0.0  0.00  1.000 -1.000000
3  0.0  0.00  0.000  0.194947
Matrices W and H:
      0     1     2     3
0  2.61  0.24  0.00  0.12
1  0.00  0.05  0.02  0.17
2  1.97  0.00  0.58  0.83
3  0.05  0.00  0.00  0.00
      0     1     2     3
0  0.38  0.65  0.34  0.41
1  0.00  1.20  0.15  3.72
2  0.42  1.09  1.38  0.07
3  0.00  0.11  0.65  0.17
RMSE of LU:  0.072
RMSE of WH:  0.071


## Data preparation for Spark ALS
1. Data preparation for Spark ALS

Let's talk about data preparation. Data preparation will consist of two things: 1. Correct dataframe format 2. Correct schema First, dataframe format.

2. Conventional Dataframe

Most dataframes you've seen probably look like this, with userId's in one column, all the features in the remaining columns, and the values of those features making up the contents of those columns. However, many Pyspark algorithms, ALS included, require your data to be in row-based format

3. Row-based data format

like this. The data is the same. The first column contains userIds, but rather than a different feature in each column, column 2 contains feature names, and column 3 contains the value of that feature for that user. So a user's data can be spread

4. Row-based data format (cont.)

across several rows, and rows contain no null values. Depending on your data, you may need to convert it to this format. Now let's talk about creating the right schema.

5. Correct schema

As you see, our userId column and our generically named column of movie titles are strings. Pyspark's implementation of ALS can only consume

6. Must be integers

userIds and movieIds as integers.So, again, you might need to convert your data to integers. Let's walk through an example of how to do all of this.

7. Conventional Dataframe

Here's a conventional dataframe. To convert it to a "long" or "dense" matrix, we will use a user-defined function called "wide_to_long":

8. Wide to long function

We won't go into the detail of how it works here, but it turns the conventional dataframe into a row-based dataframe like this:

9. Long DF Output

If you'd like to access this function directly, a link will be provided at the end of the course. So we have the right dataframe format, let's get the right schema. In order to have integer user and movieId's we need to assign unique integers to the userId's and the movieId's. To do this, we will follow 3 steps

10. Steps to get integer ID's

1. Extract unique userIds/movieIds 2. Assign unique integers to each id 3. Rejoin these unique integer id's back to the ratings data. Let's start with userIds.

11. Extracting distinct user IDs

Let's first run this query to get all the distinct userIds into one dataframe and call it users.

12. Monotonically increasing ID

Then we'll import a method called "monotonically_increasing_id()" which will assign a unique integer to each row of our users dataframe. We need to be careful when using this because it will treat each partition of data independently, meaning the same integer could be used in different partitions. In order to get around this, we'll convert our data into one partition using the coalesce method.

13. Coalesce method

Also note that while the integers will be increasing by a value of 1 over each row, they may not necessarily start at 1. That's not super important here, what's really important is that they are unique. So now we can create a new column in our users dataframe

14. Persist method

called userIntId, set it to monotonicallyIncreasingId, and we will have our new userIntegerIds. Note that the monotonically_increasing_id() method can be a bit tricky as the values it provides can change as you do different things to your dataset. For this reason, we've called the .persist() method to tell Spark to keep these values the same across all dataframe operations. We'll do the

15. Movie integer IDs

same thing with the movie id's and now we have two dataframes, one with our userIds and one with our movieIds. So let's join

16. Joining UserIds and MovieIds

them together along with our original dataframe on our userId and variable columns using the .join() method, specifying a "left" join. We can be even more thorough by creating a new dataframe with only the columns ALS needs, and renaming our columns using the .alias() method, which renames the column on which it is called. Like this:

17. Joining User and Movie Integer Ids

like this.

18. Let's practice!

Now let's prepare some data.

In [26]:
U = pd.DataFrame( columns = ['U_LF_1',  'U_LF_2' , 'U_LF_3' , 'U_LF_4'], index = ['User_1', 'User_2' , 'User_3', 'User_4', 'User_5','User_6','User_7','User_8','User_9'], data = np.array([
    [0.80  ,  0.01 ,   0.30  ,   0.8],
[    0.40  ,  0.01 ,   0.06   ,  0.2]  , 
    [0.05  ,  2.10 ,   0.01  ,   2.2],
   [ 0.30  ,  0.01 ,   0.20  ,   0.2],
   [ 0.10  ,  1.50  ,  0.90 ,    0.0],
    [0.00  ,  0.03 ,   0.40  ,   0.5],
    [0.01  ,  0.02 ,   0.66  ,   0.4],
    [0.90  ,  0.70  ,  0.00  ,   1.0],
    [1.00  ,  2.00  ,  0.04  ,   0.2]]))
P =   pd.DataFrame(columns = ['Movie_1' , 'Movie_2' , 'Movie_3' , 'Movie_4'], index = ['P_LF_1', 'P_LF_2', 'P_LF_3', 'P_LF_4'], data = np.array([
[        0.5 ,     0.1 ,     0.4  ,   1.10],
[      0.2   ,   2.0  ,    0.0   ,  0.01],
[      0.3   ,   1.9   ,   0.6  ,   0.90],
[      1.0    ,  0.2  ,    1.0 ,    0.89]  ]))

### Estimating recommendations
Use your knowledge of matrix multiplication to determine which movie will have the highest recommendation for User_3. The ratings matrix has been factorized into `U` and `P` with ALS.

**Instructions 1/3**

- View the left factor matrix, `U`, using the `print` function.

**Instructions 2/3 **

- Did you see anything interesting about `User_3`? Now inspect the right factor matrix, `P`. Use the print function.

**Instructions 3/3**

- Looking at `U` and `P`, which movie do you think will have the highest recommendation for `User_3`.
- Multiply `U` and `P` using numpy's matmul to obtain recommendations. Call this `UP`.
- Complete the code to print `UP`.

In [27]:
# View left factor matrix 1/3
print(U)

# View right factor matrix 2/3
print(P)

# Multiply factor matrices 3/3
UP = np.matmul(U,P)

# Convert to pandas DataFrame
print(pd.DataFrame(data = UP, columns = P.columns, index = U.index))

        U_LF_1  U_LF_2  U_LF_3  U_LF_4
User_1    0.80    0.01    0.30     0.8
User_2    0.40    0.01    0.06     0.2
User_3    0.05    2.10    0.01     2.2
User_4    0.30    0.01    0.20     0.2
User_5    0.10    1.50    0.90     0.0
User_6    0.00    0.03    0.40     0.5
User_7    0.01    0.02    0.66     0.4
User_8    0.90    0.70    0.00     1.0
User_9    1.00    2.00    0.04     0.2
        Movie_1  Movie_2  Movie_3  Movie_4
P_LF_1      0.5      0.1      0.4     1.10
P_LF_2      0.2      2.0      0.0     0.01
P_LF_3      0.3      1.9      0.6     0.90
P_LF_4      1.0      0.2      1.0     0.89
        Movie_1  Movie_2  Movie_3  Movie_4
User_1      NaN      NaN      NaN      NaN
User_2      NaN      NaN      NaN      NaN
User_3      NaN      NaN      NaN      NaN
User_4      NaN      NaN      NaN      NaN
User_5      NaN      NaN      NaN      NaN
User_6      NaN      NaN      NaN      NaN
User_7      NaN      NaN      NaN      NaN
User_8      NaN      NaN      NaN      NaN
User_9  

C:\Users\ahmad.alashmony\AppData\Local\Temp\ipykernel_16068\635015382.py:8: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy array (eg 'ufunc(df1, np.asarray(df2)') to keep the current behaviour, or align manually (eg 'df1, df2 = df1.align(df2)') before passing to the ufunc to obtain the future behaviour and silence this warning.
  UP = np.matmul(U,P)


### RMSE as ALS alternates
As you know, ALS will alternate between the two factor matrices, adjusting their values each time to iteratively come closer and closer to approximating the original ratings matrix. This exercise is intended to illustrate this to you.

Matrix `T` is a ratings matrix, and matrices `F1`, `F2`, `F3`, `F4`, `F5`, and `F6` are the respective products of ALS after iterating 2, 3, 4, 5, and 6 times respectively. Follow the instructions below to see how the RMSE changes as ALS iterates.

**Instructions**

- Use `getRMSE(pred_matrix, actual_matrix)` to calculate the RMSE of `F1`.
- Put `F2`, `F3`, `F4`, `F5`, and `F6` into one list called `Fs`.
- Complete the `getRMSEs(listOfPredMatrices, actualValues)` function to calculate the RMSE of each matrix in the `Fs` list.

#### fs
- F1

   0  1  2  3
0  2  4  3  3
1  1  3  2  1
2  1  4  4  3
3  1  4  4  3
4  3  3  1  3
5  4  2  4  1
6  2  4  3  4
7  4  3  3  4
8  4  1  3  2

- F2

          0         1         2         3
0  0.727685  0.000000  0.781614  0.000000
1  0.297216  0.000000  0.289833  0.149972
2  0.668410  5.133314  2.090948  1.727615
3  0.103563  0.449466  0.000000  0.000000
4  0.278805  0.000000  0.564327  0.326220
5  0.183318  0.000000  0.000000  0.229848
6  0.000000  1.817581  0.718101  0.687327
7  0.981968  2.720611  1.029219  0.000000
8  0.000000  2.667367  0.820015  0.000000

- F3

          0         1         2         3
0  1.492245  0.000000  0.990605  0.000000
1  0.397179  0.000000  0.435733  0.563462
2  0.141205  4.940441  1.881492  1.905120
3  0.109256  0.343033  0.000000  0.000000
4  0.619273  0.000000  0.686859  0.973293
5  0.377921  0.000000  0.000000  0.608201
6  0.000000  1.350676  0.676210  0.883204
7  1.391171  1.810945  1.390281  0.000000
8  0.000000  3.820890  1.160296  0.000000

- F4

          0         1         2         3
0  1.232812  0.000000  0.963432  0.000000
1  0.308675  0.000000  0.406568  0.827733
2  0.399142  4.868625  1.987510  1.673421
3  0.281845  0.419753  0.000000  0.000000
4  0.531455  0.000000  0.626833  1.098208
5  0.319177  0.000000  0.000000  0.855895
6  0.000000  1.488357  0.719290  0.807520
7  1.615384  1.553621  1.526890  0.000000
8  0.000000  4.018753  0.856822  0.000000

- F5

          0         1         2         3
0  1.191828  0.000000  0.756902  0.000000
1  0.482960  0.000000  0.466946  0.634913
2  0.270822  4.665225  2.179591  2.058923
3  0.364895  0.466308  0.000000  0.000000
4  0.703867  0.000000  0.661523  0.896099
5  0.829117  0.000000  0.000000  0.413693
6  0.000000  1.350148  0.788875  1.027619
7  1.476283  1.632972  1.457697  0.000000
8  0.000000  4.086277  0.735722  0.000000

- F6

          0         1         2         3
0  1.392534  0.000000  1.190954  0.000000
1  0.448433  0.000000  0.388912  0.654163
2  0.086018  4.628878  2.205928  2.051924
3  0.248835  0.437081  0.000000  0.000000
4  0.659538  0.000000  0.570960  0.909904
5  0.611369  0.000000  0.000000  0.815595
6  0.000000  1.338655  0.859938  0.944789
7  1.510931  1.681587  1.445032  0.000000
8  0.000000  4.214108  0.629327  0.000000

In [28]:
def getRMSE(pred, actual):
    """ 
    Compute the RMSE between pred and actual values.
    
    Parameters:
      - pred: list or array of predictions
      - actual: list or array of true values
      
    Returns: RMSE
    """
    MSE = (((pred - actual)**(2)).sum().sum()/(pred.shape[0]*pred.shape[1]))
    print ("F1: ", MSE**.5)
    
def getRMSEs(listOfPredMatrices, actualValues):
    """ 
    Computes the RMSE between predictions and actual values for various sets of predictions.
    
    Parameters:
      - pred: list of matrices each containing predictions
      - actual: array of true values
      
    Returns: Printed statement of integer index of predition and respective RMSE based on actual values provided.
    """
    for i, pred in enumerate(listOfPredMatrices):
        MSE = (((pred - actualValues)**(2)).sum().sum()/(pred.shape[0]*pred.shape[1]))
        print ("F" + str(i+2) + ":", MSE**.5)

In [29]:
'''
# Use getRMSE(preds, actuals) to calculate the RMSE of matrices T and F1.
getRMSE(F1, T)

# Create list of F2, F3, F4, F5, and F6
Fs = [F2, F3, F4, F5, F6]

# Calculate RMSE for F2, F3, F4, F5, and F6.
getRMSEs(Fs, T)
'''

'\n# Use getRMSE(preds, actuals) to calculate the RMSE of matrices T and F1.\ngetRMSE(F1, T)\n\n# Create list of F2, F3, F4, F5, and F6\nFs = [F2, F3, F4, F5, F6]\n\n# Calculate RMSE for F2, F3, F4, F5, and F6.\ngetRMSEs(Fs, T)\n'

## ALS parameters and hyperparameters
1. ALS parameters and hyperparameters

As with most algorithms, ALS has arguments that we give it and hyperparameters which must be tuned in order to generate the best predictions.

2. Example ALS model code

Here is what a built-out ALS model looks like. Let's review each argument and hyperparameter.

3. Column names

The userCol, itemCol and ratingCol are straightforward. They simply tell spark which columns in your dataframe contain the respective userIds', itemIds' and ratings. The first ALS hyperparameter is the rank.

4. Rank

As you already know, ALS will take a matrix of ratings, and it will factor that matrix into two different matrices, one representing the users, and the other representing the products, or items, or in our case, movies. In the process of doing this,

5. Rank (cont.)

latent features are uncovered. ALS allows you to choose the number of latent features that are created, which is referred to as the "rank" hyperparameter, often represented by the letter k.

6. Rank

Your objective with the data will be to determine the "rank". If you're trying to find meaningful groupings or categories of movies to see how similar or different movies are, you may want to experiment with different numbers of latent features. If you have too few or too many latent features, the groupings might be difficult to understand, so you'll want to look at different numbers of latent features and manually identify what makes the most sense. For purposes of recommendations however, the best number of latent features will be found through cross-validation.

7. MaxIter

The number of iterations, or "maxIter" simply tells ALS how many times to iterate back and forth between the factors matrices, adjusting the values to reduce the RMSE. Obviously the higher number of iterations, the longer it will take to complete, and the fewer number of iterations, the higher the risk of not fully reducing the error. So you'll have to determine what works you.

8. RegParam

Many other machine learning algorithms have a regularization parameter, often called lambda. A lambda is simply a number that is added to an error metric to keep the algorithm from converging too quickly and overfitting to the training data. The lambda for ALS in Pyspark is referred to as the "regParam".

9. Alpha

We'll talk about alpha later in the course, but suffice it to say that alpha is only used when using implicit ratings, and not used with explicit ratings.

10. Non-negative

Let's talk about the ALS arguments. As mentioned previously, there are several different factorizations that can be used to factor a matrix. The one that we are interested in is the non-negative factorization, so we set the nonnegative argument to True.

11. Cold start strategy

You might be familiar with the term coldStartStrategy already. In the context of ALS, when splitting data into test and train sets, it's possible for a user to have all of their ratings inadvertantly put into the test set, leaving nothing in the train set to be used for making a prediction. In this case, ALS can't make meaningful predictions for that user, or calculate an error metric. To avoid this, we set the coldStartStrategy to "drop" which tells Spark that when these situations arise, to not use them to calculate the RMSE, and to only use users that have ratings in both the test AND training set.

12. Implicit preferences

We also need to tell Spark whether our ratings are implicit or explicit. We do this by setting the implicitPrefs argument to True or False.

13. Sample ALS model build

Once we have a built-out model like you see here, we can fit it to training data, and then generate test predictions to see how well it performs. We can do this by

14. Fit and transform methods

calling the fit and transform methods as you see here. You'll do this yourself in subsequent exercises.

15. Let's practice!

Now it's your turn to build some models.

### Correct format and distinct users
Take a look at the `R` dataframe. Notice that it is in conventional or "wide" format with a different movie in each column. Also notice that the User's and movie names are not in integer format. Follow the steps to properly prepare this data for ALS.

**Instructions**

- Import the `monotonically_increasing_id` package from `pyspark.sql.functions` and view the `R` dataframe using the `.show()` method.
- Use the `to_long()` function to convert the `R` dataframe into a "long" data frame. Call the new dataframe `ratings`.
- Create a dataframe called users that contains all the `.distinct()` users from the dataframe and repartition the dataframe into one partition using the `.coalesce(1)` method.
- Use the `monotonically_increasing_id()` method inside of `withColumn()` to create a new column in the users dataframe that contains a unique integer for each user. Call this column `userId`. Be sure to call the `.persist()` method on the final dataframe to ensure the new integer IDs persist.

R DataFrame

|            User|Shrek|Coco|Swing Kids|Sneakers|
|----------------|-----|----|----------|--------|
|    James Alking|    3|   4|         4|       3|
|Elvira Marroquin|    4|   5|      null|       2|
|      Jack Bauer| null|   2|         2|       5|
|     Julia James|    5|null|         2|       2|


In [30]:
#R = pd.DataFrame(columns = ['User','Shrek','Coco','Swing' 'Kids','Sneakers'], data = np.array([
#['James Alking',3,4,4,3],['Elvira Marroquin',4,5,np.nan,2],['Jack Bauer',np.nan,2,2,5],['Julia James',5,np.nan,2,2]]))

#R= spark.createDataFrame(R)

R= spark.read.csv("to_long_dataset.csv", inferSchema=True, header=True)

from pyspark.sql.functions import explode, lit, struct, col
from numpy import array

def to_long(df, by = ["User"]):
    """ 
    Converts traditional or "wide" dataframe into a "row-based" dataframe, also known as a "dense" or "long" dataframe.
    
    Parameters:
      - df: array of columns with column names
      - by: name of column which serves as
      
    Returns: Row-based dataframe with no null values
    """
    cols = [c for c in df.columns if c not in by]
    # Create and explode an array of (column_name, column_value) structs
    kvs = explode(array([
      struct(lit(c).alias("Movie"), col(c).alias("Rating")) for c in cols
    ])).alias("kvs")
    return df.select(by + [kvs]).select(by + ["kvs.Movie", "kvs.Rating"]).filter("rating IS NOT NULL")

In [31]:
'''
# Import monotonically_increasing_id and show R
from pyspark.sql.functions import monotonically_increasing_id
R.show()

# Use the to_long() function to convert the dataframe to the "long" format.
ratings = to_long(R)
ratings.show()

# Get unique users and repartition to 1 partition
users = ratings.select("User").distinct().coalesce(1)

# Create a new column of unique integers called "userId" in the users dataframe.
users = users.withColumn("userId", monotonically_increasing_id()).persist()
users.show()
'''

'\n# Import monotonically_increasing_id and show R\nfrom pyspark.sql.functions import monotonically_increasing_id\nR.show()\n\n# Use the to_long() function to convert the dataframe to the "long" format.\nratings = to_long(R)\nratings.show()\n\n# Get unique users and repartition to 1 partition\nusers = ratings.select("User").distinct().coalesce(1)\n\n# Create a new column of unique integers called "userId" in the users dataframe.\nusers = users.withColumn("userId", monotonically_increasing_id()).persist()\nusers.show()\n'

### Assigning integer id's to movies
Let's do the same thing to the movies. Then let's join the new user IDs and movie IDs into one dataframe.

**Instructions**


- Use the `.select()` and the `.distinct()` methods to extract all unique Movies from the ratings dataframe.
- Repartition the movies dataframe to one partition using `coalesce()`.
- Complete the partial code provided to assign unique integer IDs to each movie. Name the new column movieId and call the `.persist()` method on the resulting dataframe.
- Join the ratings dataframe to the users dataframe and subsequently to the movies dataframe. Call the result `movie_ratings`.

#### Ratings
|            User|     Movie|Rating|
|----------------|----------|------|
|    James Alking|     Shrek|     3|
|    James Alking|      Coco|     4|
|    James Alking|Swing Kids|     4|
|    James Alking|  Sneakers|     3|
|Elvira Marroquin|     Shrek|     4|
|Elvira Marroquin|      Coco|     5|
|Elvira Marroquin|  Sneakers|     2|
|      Jack Bauer|      Coco|     2|
|      Jack Bauer|Swing Kids|     2|
|      Jack Bauer|  Sneakers|     5|
|     Julia James|     Shrek|     5|
|     Julia James|Swing Kids|     2|
|     Julia James|  Sneakers|     2|

In [32]:
'''
# Extract the distinct movie id's
movies = ratings.select("Movie").distinct() 

# Repartition the data to have only one partition.
movies = movies.coalesce(1) 

# Create a new column of movieId integers. 
movies = movies.withColumn("movieId", monotonically_increasing_id()).persist() 

# Join the ratings, users and movies dataframes
movie_ratings = ratings.join(users, "User", "left").join(movies, "Movie", "left")
movie_ratings.show()
'''

'\n# Extract the distinct movie id\'s\nmovies = ratings.select("Movie").distinct() \n\n# Repartition the data to have only one partition.\nmovies = movies.coalesce(1) \n\n# Create a new column of movieId integers. \nmovies = movies.withColumn("movieId", monotonically_increasing_id()).persist() \n\n# Join the ratings, users and movies dataframes\nmovie_ratings = ratings.join(users, "User", "left").join(movies, "Movie", "left")\nmovie_ratings.show()\n'

# Recommending Movies

## Introduction to the MovieLens dataset
1. Introduction to the MovieLens dataset

Up until now we've only been using sample datasets. Now we're going to begin using actual data using the

2. MovieLens dataset

MovieLens dataset. This dataset is made available by the good people at GroupLens.org and contains

3. MovieLens summary stats

roughly 20 million ratings for over 138,000 users and more than 27,000 movies. In order to provide you with a better learning experience, we will achieve shorter runtimes by using a subset of the original dataset including 100,000 ratings. In addition to the ratings data, Grouplens.org also provides additional datafiles that include information on movie genres and other types of tags that movie watchers have provided for them. We'll take what you've learned from the previous chapters and explore the data, prepare the data, build out a cross-validated ALS model, generate predictions and assess the model's performance. First we'll view the data using the

4. Explore the data

.show() and .columns() methods, as well as some other methods to understand the nature of the dataset.

5. MovieLens sparsity

Then we'll calculate it's sparsity using this sparsity formula, and then we'll assess whether further preparation is needed in order to adequately prepare it for ALS. If you're not familiar with the term sparsity, it simply provides a measure of how empty a matrix is, or what percentage of the matrix is empty. In essence, this formula is simply the number of ratings that a matrix contains divided by the number of ratings it could contain given the number of users and movies in the matrix.

6. Sparsity: numerator

The code to calculate sparsity is pretty straightforward. We'll simply get the numerator by counting the number of ratings in the ratings dataframe

7. Sparsity: users and movies

then we'll get the number of distinct users and the number of distinct items or movies.

8. Sparsity: denominator

We'll then multiply the number of users and number of movies together to get the denominator

9. Sparsity

and simply divide the numerator by the denominator, and substract the result from 1. Because division in Pyhton will return an integer, we multiply the numerator by 1.0 to ensure a decimal or float is returned. Let's go over some other techniques that may or may not be new to you.

10. The .distinct() method

As you may already know, the .distinct() method simply returns all the unique values in a column. For example, if you want to know how many unique users there are in a table, you could simply select the userId column from the dataframe, then run the distinct and count methods like you see here.

11. GroupBy method

The groupBy method organizes data by the unique values of a specific column to return subtotals for those unique values. For example{{1}}, if you wanted to look at total number of ratings each user has provided you would first need to groupBy userId as you see here, then

12. GroupBy method

call the count method as you see here. With this, you could then

13. GroupBy method min

get the min

14. GroupBy method max

or max

15. GroupBy method avg

or average of that same column.

16. Filter method

The filter method allows you to filter out any data that doesn't meet your specified criteria. {{1}}For example if you wanted to only consider users that have rated at least 20 movies, you would simply apply the same groupby and count methods, and then add a filter method specifying that the count column should only include values greater than 20.

17. Let's practice!

Let's apply what you've learned to a real data set.

### Build out an ALS model
Let's specify your first ALS model. Complete the code below to build your first ALS model.

Recall that you can use the `.columns` method on the ratings data frame to see what the names of the columns are that contain user, movie, and ratings data. Spark needs to know the names of these columns in order to perform ALS correctly.

**Instructions**

- Before building our ALS model, we need to split the data into training data and test data. Use the `randomSplit()` method to split the ratings dataframe into `training_data` and `test_data` using an 0.8/0.2 split respectively and a seed for the random number generator of `42`.
- Tell Spark which columns contain the `userCol`, `itemCol` and `ratingCol`. Use the `.columns` method if needed. Complete the hyperparameters. Set the `rank` to 10, the `maxIter` to 15, the `regParam` or lambda to .1, the `coldStartStrategy` to "drop", the `nonnegative` argument should be set to True, and since our data contains explicit ratings, set the `implicitPrefs` argument to False.
- Now fit the als model to the `training_data` portion of the ratings data by calling the `als.fit()` method on the `training_data` provided. Call the fitted model model.
- Generate predictions on the `test_data` portion of the ratings data by calling the `model.transform()` method on the `test_data` provided. Call the predictions `test_predictions`. Feel free to view the predictions by calling the `.show()` method on the `test_predictions`


In [33]:
ratings = spark.read.csv('toy_ratings.csv', inferSchema=True, header=True)
ratings.count()

13

In [34]:
# Split the ratings dataframe into training and test data
(training_data, test_data) = ratings.randomSplit([0.8, 0.2], seed=42)

# Set the ALS hyperparameters
from pyspark.ml.recommendation import ALS
als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating", rank =10, maxIter =15, regParam =0.1,
          coldStartStrategy="drop", nonnegative =True, implicitPrefs = False)

# Fit the mdoel to the training_data
model = als.fit(training_data)

# Generate predictions on the test_data
test_predictions = model.transform(test_data)
test_predictions.show()

C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:132: UserWarning: A NumPy version >=1.21.6 and <1.28.0 is required for this version of SciPy (detected version 1.20.1)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|     2|      0|     3| 1.7681035|
|     2|      2|     4| 1.2946719|
|     0|      3|     4| 3.8376758|
+------+-------+------+----------+



### Build RMSE evaluator
Now that you know how to fit a model to training data and generate test predictions, you need a way to evaluate how well your model performs. For this we'll build an evaluator. Evaluators in Spark can be built out in various ways. For our purposes, we want a `regressionEvaluator` that calculates the RMSE. After we build our `regressionEvaluator`, we can fit the model to our data and generate predictions.

**Instructions**

- Import the required `RegressionEvaluator` package from the `pyspark.ml.evaluation` class.
- Complete the evaluator code, specifying the `metric` name to be "rmse". Set the `labelCol` to the name of the column in our ratings data that contains our ratings (use the `ratings.columns` method to see column names) and set the `prediction` column name to "prediction".
- Confirm that the evaluator was properly created by extracting each of the three parameters from it. Do this by running the following 3 lines of code, each within a print statement:
    - `evaluator.getMetricName()`
    - `evaluator.getLabelCol()`
    - `evaluator.getPredictionCol()`

In [35]:
# Import RegressionEvaluator
from pyspark.ml.evaluation import RegressionEvaluator

# Complete the evaluator code
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")

# Extract the 3 parameters
print(evaluator.getMetricName())
print(evaluator.getLabelCol())
print(evaluator.getPredictionCol())

rmse
rating
prediction


### Get RMSE
Now that you know how to build a model and generate predictions, and have an evaluator to tell us how well it predicts ratings, we can calculate the RMSE to see how well an ALS model performed. We'll use the evaluator that we built in the previous exercise to calculate and print the rmse.

**Instructions**

- Call the `.evaluate()` method on our evaluator to calculate our RMSE on the `test_predictions` dataframe. Call the result `RMSE`.
- Print the `RMSE`

In [36]:
# Evaluate the "test_predictions" dataframe
RMSE = evaluator.evaluate(test_predictions)

# Print the RMSE
print (RMSE)

1.7187901116515298


## ALS model buildout on MovieLens Data
1. ALS model buildout on MovieLens Data

If you remember from the last chapter, you built out a model on the ratings dataset. The code looked like this:

2. Fitting a basic model

Now, the RMSE that you got was lower than the 1.45 shown here. But what if you went through this whole process and got an error metric that you weren't satisfied with, like this RMSE of 1.45. You might want to try other combinations of hyperparameter values to try and reduce that. Spark makes it easy to do this by using two additional tools called

3. Intro to ParamGridBuilder and CrossValidator

the ParamGridBuilder and the CrossValidator. These tools will allow you to try many different hyperparameter values and have Spark identify the best combination. Let's talk about how to use them.

4. ParamGridBuilder

The ParamGridBuilder tells Spark all the hyperparameter values you want it to try. To do this, we first import the ParamGridBuilder package, instantiate it and give it a name. We'll call it param_grid. We then add each hyperparameter name calling the .addGrid()

5. Adding Hyperparameters to the ParamGridBuilder

method on our als algorithm and hyperparameter name as you see here. Notice the empty lists to the right of the hyperparameter names. This is where we input the values we want Spark to try for each hyperparameter, like this:

6. Adding Hyperparameter Values to the ParamGridBuilder

Once we've added all of this, we call the .build() method to complete the build of our param_grid. Now let's look at the CrossValidator.

7. CrossValidator

The CrossValidator essentially fits a model to several different portions of our training dataset called folds, and then generates predictions for each respective holdout portion of the dataset to see how it performs.

8. CrossValidator instantiation and estimator

To properly use the CrossValidator, we first import the CrossValidator package, instantiate a CrossValidator and give it a name, we'll call it cv here.

9. CrossValidator ParamMaps

We then tell it to use our als model as an estimator by setting estimator argument equal to the name of our model which is als. We'll set the estimatorParamMaps to our param_grid that we built so that Spark knows what values to try as it works to identify the best combination of hyperparameters. Then we provide the name of our evaluator so it knows how to measure each model's performance by simply setting the evaluator argument to the name of our evaluator which is "evaluator".

10. CrossValidator

We finish by setting the numFolds argument to the number of times we want Spark to test each model on the training data, in this case, 5 times. Let's go over how to integrate these into a full code buildout.

11. Random split

We'll first split our data into training and test sets using the randomSplit() method and we'll build a generic ALS model without any hyperparameters, only the model parameters as you see here. The cross validator will take care of the hyperparameters.

12. ParamGridBuilder

We'll build our ParamGridBuilder so Spark knows what hyperparameter values to test.

13. Evaluator

We'll create an evaluator so Spark knows how to evaluate each model.

14. CrossValidator

Then the CrossValidator will tell Spark the algorithm, the hyperparameters and values, and the evaluator to use to find the best model, and the number of training set folds we want each model to be tested on.

15. Best model

We then fit our CrossValidator on the training data to have Spark try all the combindations of hyperparameters we specified by calling the cv.fit() method on the training data. Once it's finished running, we extract the best-performing model by calling the bestModel() method on our model. We'll call this our best_model and

16. Predictions and performance evaluation

with it, we can generate predictions on the test set, print the error metric and the respective hyperparameter values using the code you see here. And now we have our cross-validated model.

17. Let's practice!

Let's build a real model on a real dataset.

### Viewing the MovieLens Data
Familiarize yourself with the ratings dataset provided here. Would you consider the data to be implicit or explicit ratings?

**Instructions**

- Look at the `.columns` of the ratings dataframe.
- Look at the first few rows of ratings dataframe using the `.show()` method.

In [37]:
ratings = spark.read.csv('ratings.csv', inferSchema = True, header = True)

In [38]:
# Look at the column names
print(ratings.columns)

# Look at the first few rows of data
print(ratings.show())

['userId', 'movieId', 'rating', 'timestamp']
+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|     31|   2.5|1260759144|
|     1|   1029|   3.0|1260759179|
|     1|   1061|   3.0|1260759182|
|     1|   1129|   2.0|1260759185|
|     1|   1172|   4.0|1260759205|
|     1|   1263|   2.0|1260759151|
|     1|   1287|   2.0|1260759187|
|     1|   1293|   2.0|1260759148|
|     1|   1339|   3.5|1260759125|
|     1|   1343|   2.0|1260759131|
|     1|   1371|   2.5|1260759135|
|     1|   1405|   1.0|1260759203|
|     1|   1953|   4.0|1260759191|
|     1|   2105|   4.0|1260759139|
|     1|   2150|   3.0|1260759194|
|     1|   2193|   2.0|1260759198|
|     1|   2294|   2.0|1260759108|
|     1|   2455|   2.5|1260759113|
|     1|   2968|   1.0|1260759200|
|     1|   3671|   3.0|1260759117|
+------+-------+------+----------+
only showing top 20 rows

None


### Calculate sparsity
As you know, ALS works well with sparse datasets. Let's see how much of the ratings matrix is actually empty.

Remember that sparsity is calculated by the number of cells in a matrix that contain a rating divided by the total number of values that matrix could hold given the number of users and items (movies). In other words, dividing the number of ratings present in the matrix by the product of users and movies in the matrix and subtracting that from 1 will give us the sparsity or the percentage of the `ratings` matrix that is empty.

**Instructions**

- Calculate the `numerator` of the sparsity metric by counting the total number of ratings that are contained in the ratings matrix.
- Calculate the number of `distinct()` `userIds` and `distinct()` `movieIds` in the ratings matrix.
- Calculate the denominator of the sparsity metric by multiplying the number of users by the number of movies in the ratings matrix.
- Calculate and print the sparsity by dividing the `numerator` by the `denominator`, subtracting from 1 and multiplying by 100. The 1.0 is added to ensure the `sparsity` is returned as a decimal and not an integer.

In [39]:
# Count the total number of ratings in the dataset
numerator = ratings.select("rating").count()

# Count the number of distinct userIds and distinct movieIds
num_users = ratings.select("userId").distinct().count()
num_movies = ratings.select("movieId").distinct().count()

# Set the denominator equal to the number of users multiplied by the number of movies
denominator = num_movies * num_users

# Divide the numerator by the denominator
sparsity = (1.0 - (numerator *1.0)/denominator)*100
print("The ratings dataframe is ", "%.2f" % sparsity + "% empty.")

The ratings dataframe is  98.36% empty.


### The GroupBy and Filter methods
Now that we know a little more about the dataset, let's look at some general summary metrics of the ratings dataset and see how many ratings the movies have and how many ratings each users has provided.

Two common methods that will be helpful to you as you aggregate summary statistics in Spark are the `.filter()` and the `.groupBy()` methods. The `.filter()` method allows you to filter out any data that doesn't meet your specified criteria.

**Instructions**

- Import `col` from the `pyspark.sql.functions`, and view the ratings dataset using the `.show()`.
- Apply the `.filter()` method on the ratings dataset with the following filter inside the parenthesis in order to include only `userId`'s less than 100: `col("userId") < 100`.
- Call the `.groupBy()` method on the ratings dataset to group the data by `userId`. Call the `.count()` method to see how many movies each `userId` has rated.

In [40]:
# Import the requisite packages
from pyspark.sql.functions import col

# View the ratings dataset
ratings.show()

# Filter to show only userIds less than 100
ratings.filter(col("userId") < 100).show()

# Group data by userId, count ratings
ratings.groupBy("userId").count().show()

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|     31|   2.5|1260759144|
|     1|   1029|   3.0|1260759179|
|     1|   1061|   3.0|1260759182|
|     1|   1129|   2.0|1260759185|
|     1|   1172|   4.0|1260759205|
|     1|   1263|   2.0|1260759151|
|     1|   1287|   2.0|1260759187|
|     1|   1293|   2.0|1260759148|
|     1|   1339|   3.5|1260759125|
|     1|   1343|   2.0|1260759131|
|     1|   1371|   2.5|1260759135|
|     1|   1405|   1.0|1260759203|
|     1|   1953|   4.0|1260759191|
|     1|   2105|   4.0|1260759139|
|     1|   2150|   3.0|1260759194|
|     1|   2193|   2.0|1260759198|
|     1|   2294|   2.0|1260759108|
|     1|   2455|   2.5|1260759113|
|     1|   2968|   1.0|1260759200|
|     1|   3671|   3.0|1260759117|
+------+-------+------+----------+
only showing top 20 rows

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|     31|   2.5|126075

### MovieLens Summary Statistics
Let's take the `groupBy()` method a bit further.

Once you've applied the `.groupBy()` method to a dataframe, you can subsequently run aggregate functions such as `.sum()`, `.avg()`, `.min()` and have the results grouped. This exercise will walk you through how this is done. The `min` and `avg` functions have been imported from `pyspark.sql.functions` for you.

**Instructions**

- Group the data by `movieId` and use the `.count()` method to calculate how many ratings each movie has received. From there, call the `.select()` method to select the following metrics:
    - `min("count")` to get the smallest number of ratings that any movie in the dataset. This first one is given to you as an example.
    - `avg("count")` to get the average number of ratings per movie
Do the same thing, but this time group by userId to get the min and avg number of ratings.

In [41]:
from pyspark.sql.functions import min, avg

In [42]:
# Min num ratings for movies
print("Movie with the fewest ratings: ")
ratings.groupBy("movieId").count().select(min("count")).show()

# Avg num ratings per movie
print("Avg num ratings per movie: ")
ratings.groupBy("movieId").count().select(avg("count")).show()

# Min num ratings for user
print("User with the fewest ratings: ")
ratings.groupBy("userId").count().select(min("count")).show()

# Avg num ratings per users
print("Avg num ratings per user: ")
ratings.groupBy("userId").count().select(avg("count")).show()

Movie with the fewest ratings: 
+----------+
|min(count)|
+----------+
|         1|
+----------+

Avg num ratings per movie: 
+------------------+
|        avg(count)|
+------------------+
|11.030664019413193|
+------------------+

User with the fewest ratings: 
+----------+
|min(count)|
+----------+
|        20|
+----------+

Avg num ratings per user: 
+------------------+
|        avg(count)|
+------------------+
|149.03725782414307|
+------------------+



#### View Schema
As you know from previous chapters, Spark's implementation of ALS requires that `movieIds` and `userIds` be provided as `integer` datatypes. Many datasets need to be prepared accordingly in order for them to function properly with Spark. A common issue is that Spark thinks numbers are strings, and vice versa.

Here, you'll use the `.cast()` method to address these types of problems. Let's take a look at the schema of the dataset to ensure it's in the correct format.

**Instructions**

- Use `.printSchema()` to check whether the ratings dataset contains the proper data types for ALS to function correctly. Are the `userIds` and `movieIds` provided as integer datatypes? Are the ratings in numeric format?
- Ensure that the columns of the ratings dataframe are the correct data types. Call the `cast()` method on each column and specify the `userID` and `movieId` columns to be type "integer" and the rating column to be of type "double". (We don't need the timestamp column, so we can leave that out.)
- Call `.printSchema()` again on `ratings` to confirm that the data types are now correct.

In [43]:
# Use .printSchema() to see the datatypes of the ratings dataset
ratings.printSchema()

# Tell Spark to convert the columns to the proper data types
ratings = ratings.select(ratings.userId.cast("integer"), ratings.movieId.cast("integer"), ratings.rating.cast("double"))

# Call .printSchema() again to confirm the columns are now in the correct format
ratings.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)



## Model Performance Evaluation
1. Model Performance Evaluation and Output Cleanup

Congratulations. You just built your first cross-validated ALS model. Now let's determine whether the model suits your needs or not. The primary way to do this is to examine

2. Root mean squared error

the error metric, in this case, the RMSE. The RMSE tells us, on average, how far a given prediction is from it's corresponding actual value.

3. Pred vs actual

It's fairly straightforward. If we have predictions and actual values,

4. Pred vs actual: difference

the RMSE subtracts she actual value from the prediction,

5. Difference squared

then squares those differences to make them positive.

6. Sum of difference squared

It then sums those differences,

7. Average of difference squared

takes the average by dividing by the number of observations, in this case N = 4,

8. RMSE

and it then takes the square root to undo the squaring of the values that we did previously. So if we have an RMSE of .61, then on average, our predictions are either .61 above or below the original rating. Another way to evaluate our model is to look at it's recommendations. Remember, however, that ALS is often used to identify patterns and uncover latent features that are unobservable by humans, meaning that ALS can sometimes see things that may not initially make sense to us as humans. Bear this in mind as you move forward. To generate recommendations, we will use the native Spark function

9. Recommend for all users

recommendForAllUsers() which generates the top recommendations for all users. ALS recommendation output has two challenges that need to be addressed. The first is that it is in a format like

10. Unclean recommendation output

this which is perfectly usable in Pyspark, but isn't very human-readable. To resolve this, we save the dataframe as a temporary table and use

11. Cleaning up recommendation output

this sql query to make it readable. The explode command essentially takes an array like our recommendation column and separates each item within it, like this:

12. Explode function

Notice that only one movieId and it's respective recommendation value for each user is contained on each line, where previously, all recommendations for a given user were contained on one line. Also notice that ALS conveniently includes the movieId and rating column names with each value on each line. This makes it easy to separate them into different columns.

13. Adding lateral view

Adding the LATERAL VIEW to the explode function allows us to treat the exploded column as a table, and extract the individual values as separate columns. We first name the lateral view, in this case we call it exploded_table and then give it a formal table name which we call movieIds_and_ratings. This allows us to SELECT userId, and then get the movieId and ratings by referencing the movieIds_and_ratings table in the beginning of our query. The output is now readable:

14. Explode and lateral view together

And if we join it to the original movie information,

15. Joining clean recs with movie info

we can see what's going on even better. The other challenge with these recommendations is that they include predictions for movies that have been already been watched. Remember how ALS creates two factor matrices that are multiplied together to produce an approximation of the original ratings matrix. That's essentially what the ALS output is, including all movies for all users, whether they've seen them or not. A simple way to address this is to filter out the movies that have already been seen. Since we already have our clean recommendations, and the original movie_ratings,

16. Filtering recommendations

we can simply join these two dataframes together on "userId" and "movieId" using a "left" join.

17. Filtering recommendations (cont.)

The movies that have already been seen are those that have a rating from the original movie_ratings dataframe,

18. Filtering recommendations (cont.)

so if we simply add a filter so that the "rating" column of the movie_ratings dataframe is null, we'll only have predictions for movies that the individual users haven't seen.

19. Let's practice!

Now let's evaluate your model.

### Create test/train splits and build your ALS model
You already know how to build an ALS model, having done it in the previous chapter. We will do that again, but we'll take some additional steps to fully build out a cross-validated model.

First, let's import the requisite functions and create our train and test data sets in preparation for the cross validation step.

**Instructions**

- Import the `RegressionEvaluator` from `ml.evaluation`, the ALS algorithm from `ml.recommendation`, and the `ParamGridBuilder` and the `CrossValidator` from `ml.tuning`.
- Create an .80/.20 train/test split on the ratings data using the `randomSplit` method. Name the datasets `train` and `test`, and set the random seed to `1234`.
- Build out the ALS model, telling Spark the names of the columns in the ratings dataframe that correspond to the `userCol`, `itemCol` and `ratingCol`. Set the `nonnegative` argument to `True`, the `coldStartStrategy` to `"drop"` and let Spark know that these are not `implicitPrefs` by setting the `implicitPrefs` argument to `False`. Call this model `als`.
- Verify that the model was created by calling the `type()` function on `als`. The output should indicate what type of model it is.

In [44]:
# Import the required functions
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Create test and train set
(train, test) = ratings.randomSplit([0.8, 0.2], seed = 1234)

# Create ALS model
als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating", nonnegative = True, implicitPrefs = False, coldStartStrategy = "drop")

# Confirm that a model called "als" was created
type(als)

pyspark.ml.recommendation.ALS

### Tell Spark how to tune your ALS model
Now we'll need to create a ParamGrid to tell Spark what hyperparameters we want it to tune, how to tune them, and then build out an evaluator so Spark can know how to measure the algorithm's performance.

**Instructions**

- Import `RegressionEvaluator` from `pyspark.ml.evaluation` and `ParamGridBuilder` and `CrossValidator` from `pyspark.ml.tuning`.
- Build a `ParamGrid` called `param_grid` using the `ParamGridBuilder` provided. Call the `.addGrid()` method on each hyperparameter by providing the name of the model and the name of each hyperparameter (ex: `.addGrid(als.rank, [])`. Do this for the `rank`, `maxIter` and `regParam` hyperparameters. Also provide the respective lists of hyperparameter values that Spark should try, as provided here:
    - `rank: [10, 50, 100, 150]`
    - `maxIter: [5, 50, 100, 200]`
    - `regParam: [.01, .05, .1, .15]`
- Create a `RegressionEvaluator` called `evaluator`. Set the `metricName` to `"rmse"`, set the `labelCol` to `"rating"`, and tell Spark that when it generates predictions to call the `predictionCol` `"prediction"`.
- Run `len(param_grid)` to confirm that the `param_grid` was created and to confirm that the right number of hyperparameter combinations will be tested. It should be equal to the number of `rank` values * the number of `maxIter` values * the number of `regParam` values in the `ParamGridBuilder`.

In [45]:
# Import the requisite items
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Add hyperparameters and their respective values to param_grid
param_grid = ParamGridBuilder() \
            .addGrid(als.rank, [10, 50, 100, 150]  ) \
            .addGrid(als.maxIter, [5, 50, 100, 200]  ) \
            .addGrid(als.regParam,  [.01, .05, .1, .15]  ) \
            .build()
           
# Define evaluator as RMSE and print length of evaluator
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction") 
print ("Num models to be tested: ", len(param_grid))

Num models to be tested:  64


### Build your cross validation pipeline
Now that we have our data, our train/test splits, our model, and our hyperparameter values, let's tell Spark how to cross validate our model so it can find the best combination of hyperparameters and return it to us.

**Instructions**

- Create a `CrossValidator` called `cv` with our `als` model as the estimator, setting `estimatorParamMaps` to the `param_grid` you just built. Tell Spark that the `evaluator` to be used is the `"evaluator"` we built previously. Set the `numFolds` to 5.
- Confirm that our `cv` was built by printing `cv`.

In [46]:
# Build cross validation using CrossValidator
cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=5)

# Confirm cv was built
print(cv)

CrossValidator_eba568950944


### Best Model and Best Model Parameters
Now that we have our cross validator, `cv`, built out, we can tell Spark to take our data, fit the ALS algorithm to it, and try the different combinations of hyperparameter values from our `param_grid` so that it can identify what values provide the smallest RMSE. Unfortunately, this takes too long to complete here, but for your reference, this is how it is done:

```
#Fit cross validator to the 'train' dataset
model = cv.fit(train)

#Extract best model from the cv model above
best_model = model.bestModel
```

This code has been run separately, and the `best_model` has been identified and saved for you to use. Use the commands given to extract the parameters of the model.

**Instructions**

- Print type(`best_model`) to confirm that the model ALS built from our hyperparameter options is indeed completed. A print statement is needed here in order to work with subsequent print statements.
- Extract the `rank` from the `best_model` by calling the `.getRank()` method on the `best_model`.
- Extract the `maxIter` from the `best_model` by calling the `.getMaxIter()` method on the `best_model`.
- Extract the `regParam` from the `best_model` by calling the `.getRegParam()` method on the `best_model`.

In [48]:
#Fit cross validator to the 'train' dataset
model = cv.fit(train)

#Extract best model from the cv model above
best_model = model.bestModel

ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

In [ ]:
# Print best_model
print(type(best_model))

# Complete the code below to extract the ALS model parameters
print("**Best Model**")

# Print "Rank"
print("  Rank:", best_model.getRank())

# Print "MaxIter"
print("  MaxIter:", best_model.getMaxIter())

# Print "RegParam"
print("  RegParam:", best_model.getRegParam())

# What if you don't have customer ratings?

## Introduction to the Million Songs Dataset
1. Introduction to the Million Songs Dataset

By now you should be pretty comfortable with ALS. So far, we've only used explicit ratings. In most real-life situations, however, explicit ratings aren't available, and you'll have to get creative in building these types of models. One way to get around this is to use implicit ratings. Remember that while

2. Explicit vs implicit

explicit ratings are explicitly provided by users in various forms, implicit ratings are data used to infer ratings. For example, if a news website sees that in the last month you clicked on

3. Explicit vs implicit (cont.)

21 geopolitical articles and only 1 local news article, ALS can convert these numbers into scores indicating how confident it is that you like them. This approach assumes that the more you do something, the more you prefer it.

4. Implicit refresher II

ALS can use these confidence ratings to generate recommendations and you're going to learn how to do this. First, let's talk about the dataset you will be using.

5. Introduction to the Million Songs Dataset

The dataset this time comes from the Million Songs Dataset available from LabROSA at Columbia University. You're going to be using one file of this dataset called The Echo Nest Taste profile dataset. It contains information on over 1 million users including the number of times they've played nearly 400,000 songs. This is more data than we can use for this course, so we will only be using a portion of. We'll first examine the data, get summary statistics, and then build and evaluate our model. One thing to note here is that because the use of implicit ratings causes ALS to calculate a level of confidence that a user likes a song based on the number of times they've played it, the matrix will need to include zeros for the songs that each user has not yet listened to. In case your data doesn't already include the zeros, we'll walk through how to do this.

6. Add zeros sample

Let's say we have a ratings dataframe like this:

7. Cross join intro

You can use the .distinct() method to extract the unique userId's and songId's, like this.

8. Cross join output

You can then performs a cross join which joins each user to each song like this: Notice that the 3 users and 3 songs we originally had now create 9 unique pairs. Using a left join,

9. Joining back original ratings data

you can take that cross_join table, and join it with the original ratings to get the num_plays column. Notice it joins on both userId and songId. And because we want 0's in place of the null values, so that every user has a value for every song, we simply call the

10. Filling in with zero

.fillna() method telling Spark to fill the null values with 0. And you have your final product to feed to ALS.

11. Add zeros function

Here are all those steps in a clean function.

12. Let's practice!

Let's do this with our Million Songs dataset.

### Generate predictions and calculate RMSE
Now that we have a model that is trained on our data and tuned through cross validation, we can see how it performs on the test dataframe. To do this, we'll calculate the RMSE.

As a side note, the generation of test predictions takes more than a few minutes with this dataset. For this reason, the test predictions have been generated already and are provided here as a dataframe called `test_predictions`. For your reference, they are generated using this code: `test_predictions = best_model.transform(test)`.

**Instructions**

- The dataframe `test_predictions` contains predictions that our cross-validated ALS model generated using the test set that we created previously. Use the `.show()` method to take a look at it and see if the predictions seem close.
- Use the evaluator that you built previously to calculate the RMSE by calling the `.evaluate()` method on the `test_predictions` generated. Call this `RMSE`.
- Print the `RMSE`.

In [ ]:
test_predictions = best_model.transform(test)

In [ ]:
# View the predictions 
test_predictions.show()

# Calculate and print the RMSE of test_predictions
RMSE = evaluator.evaluate(test_predictions)
print(RMSE)

### Interpreting the RMSE
This model was able to achieve an `RMSE` of `0.633`. Click on the best interpretation of what this means.

Answer the question

- An RMSE of 0.633 means that the predictions are 6.33% off from the original values of the ratings matrix.

- An RMSE of 0.633 means that 6.33% of the time, the recommendations generated by our ALS model will be wrong.

- AN RMSE of 0.633 means that 6.33% of our users will receive recommendations that they won't take.

- **An RMSE of 0.633 means that on average the model predicts 0.633 above or below values of the original ratings matrix.**

### Do recommendations make sense
Now that we have an understanding of how well our model performed, and have some confidence that it will provide recommendations that are relevant to users, let's actually look at recommendations made to a user and see if they make sense.

The original ratings data is provided here as `original_ratings`. Take a look at user 60 and user 63's original ratings, and compare them to what ALS recommended for them. In your opinion, are the recommendations consistent with their original preferences?

**Instructions**

- Use the `.filter()` on the `original_ratings` dataframe to ensure that `col("userId")` equals `60` to look at user `60`'s original ratings. Call the `.sort()` method to sort the output by rating and set the `ascending` argument to `False` to see the highest ratings first.
- Use the `.filter()` and `.show()` methods on the recommendations dataset to look at user `60`'s ratings. Note that when ALS generates recommendations, they are provided in descending order by `userId`, so there's no need to sort the dataframe.
- Do the same thing for user `63`.
- Do the recommendation genres have some consistency with each user's original ratings?

In [ ]:
# Look at user 60's ratings
print("User 60's Ratings:")
original_ratings.filter(col("userId") == 60).sort("rating", ascending = False).show()

# Look at the movies recommended to user 60
print("User 60s Recommendations:")
recommendations.filter(col("userId") == 60).show()

# Look at user 63's ratings
print("User 63's Ratings:")
original_ratings.filter(col("userId") == 63).sort("rating", ascending = False).show()

# Look at the movies recommended to user 63
print("User 63's Recommendations:")
recommendations.filter(col("userId") == 63).show()

## Evaluating implicit ratings models
1. Evaluating implicit ratings models

Now that we have an implicit ratings dataset, let's discuss these types of models. The first thing you should know is that implicit ratings models have an additional hyperparameter called alpha. Alpha is an integer value that tells Spark how much each additional song play should add to the model's confidence that a user actually likes a song. Like the other hyperparameters, this will need to be tuned through cross validation. The challenge of these models is the evaluation. With explicit ratings, we used was the RMSE. It made sense in that situation because we could

2. Why RMSE worked before

match predictions back to a true measure of user preference. In the case of implicit ratings however,

3. Why RMSE doesn't work now

we don’t have a true measure of user preference. We only have the number of times a user listened to a song and a measure of how confident our model is that that they like that song. These aren't the same thing and calculating an RMSE between them doesn't make sense. However, using a test set, we can see if our model is giving high predictions to the songs that users have actually listened to. The logic being that if our model is returning a high prediction for a song that the respective user has actually listened to, then the predictions make sense, especially if they've listened to it more than once. We can measure this using this

4. (ROEM) Rank Ordering Error Metric

Rank Order Error Metric. In essence this metric checks to see if songs with higher numbers of plays have higher predictions.

5. ROEM bad predictions

For example, here is a set of bad predictions. The perc_rank column has ranked the predictions for each individual user such that the lowest prediction is in the highest precentile and the highest prediction is in the lowest percentile. Notice that these bad predictions include low predictions and high predictions for songs with more than one play indicating that the predictions may not be any better than random. If we multiply the number of plays by the percentRank, we get

6. ROEM: PercRank * plays

this np*rank column.

7. ROEM: bad predictions

When we sum that column we get our ROEM numerator{{1}}, and the sum of the numPlays column gives us our ROEM denominator. Using these, we can calculate our ROEM{{3}} to be 0.556. Values close to .5 indicate that they aren't much better than random. If we were to look at good predictions where the model gave high predictions to songs that had more than 1 play, they might look like this:

8. Good predictions

Notice that songs that have been played have high ratings indicating that the predictions are better than random. Which subsequently gives us an ROEM of

9. ROEM: good predictions

0.1111. This is much closer to 0, where we want to be. Unfortunately Spark hasn't implemented an evaluator metric like this, so you'll need to build it manually. An ROEM function will be provided to you in subsequent exercises. And for your reference, the code to build it is provided at the end of this course

10. ROEM: link to function on GitHub

Using this function, and a for loop

11. Building several ROEM models

you can build several models as you see here, each with different hyperparameter values. You'll want to create a model for each combination of hyperparameter values that you want to try.

12. Error output

You can then fit each one to the training data, extract each model's test predictions, and then calculate the ROEM for each one. This is a simplified way to do this. Full cross-validation is imperative to building good models. It is beyond the scope of this course to teach how to code a function that manually cross-validates and evaluates models like this, but doing so should be done, and code to do so is provided at the end of the course.

13. Let's practice!

Let's put this into practice.

## Overview of binary, implicit ratings
1. Overview of binary, implicit ratings

So far we've covered sitautions when you have explicit ratings, and when you have implicit ratings from user behavior counts. Now we're going to cover the situation when you might not even have behavior counts. In some situations, you may only have binary data that tells you whether a user has or has not taken an action with no indication of how many times they've done so. To go back to the movie example, if you know whether customers have watched certain movies, but don't have information on how many times or how much they actually liked them, you could simply feed binary data to ALS that indicates which customers have watched each movie and which ones haven't. ALS can still pull signal from this type of data and make meaningful predictions. When taking this approach, the data will look like this.

2. Binary ratings

Notice that all ratings are either a 1 or a 0. We must treat binary ratings like these as implicit ratings. If we treated them like explicit ratings and didn't include the 0's, the best performing model would simply predict 1 for everything, and deliver a deceivingly ideal RMSE of 0. Also, as with our previous Million Songs model, we can't use the RMSE as a model evaluation metric. Ultimately, when our machine learning process holds out random observations in the test set, we want our model to generate high predictions for those movies that users have actually watched. For this reason, we'll use our ROEM metric again. We'll apply the same concepts we've covered previously on this binary dataset. The convenience of using the MovieLens dataset is that we can see how our binary model performs against the original, true preference ratings of the original MovieLens dataset.

3. Class imbalance

One word about binary models. While it's perfectly feasible to feed binary data like this into ALS and get meaningful recommendations, the data does have a sort of class imbalance where the vast majority of ratings are 0's with a small percentage of 1's. Since implicit ratings models use customized error metrics like ROEM and not RMSE, the class imbalance doesn't really pose a problem like it might in classification problems. ALS can still generate meaningful recommendations from this type of data but there are strategies that can be taken with the data to try and improve recommendations.

4. Item weighting

For example, rather than treat unseen movies purely as 0's, you can weight them higher if more people have seen them. This assumes that if many people have seen a movie, it must be a pretty good movie and therefore should be treated with a little more weight, and vice versa. This is called item weighting.

5. Item weighting and user weighting

Likewise you could weight movies by individual user behavior. For example, if a user has seen lots of movies, you could weight their unseen movies lower assuming that if a user has seen lots of movies, they know what they like and have deliberately chosen NOT to view the movies they haven't seen and therefore those movies deserve a lower weighting. While these methods are applicable, their methods haven't been implemented into the Pyspark framework, and therefore require a lot of manual work which is beyond the scope of this course. However, if you'd like to learn more about these types of approaches, you can read the paper referenced at the end of the course.

6. Let's practice!

Let's build a binary ratings model.

# Course recap
1. Course recap

Congratulations. You've now completed this course on building Collaborative Filtering recommendation engines in Pyspark. We've covered a number of things from why these are important to matrix multiplication and factorization and latent features. But most importantly, you've learned how to build and interpret recommendation engines with three different data types:

2. Course summary

* Explicit Ratings * Implicit Ratings using user behavior counts{{1}} * Implicit Ratings using binary user behavior{{2}} With this information you'll be well-prepared to build a collaborative-filtering recommendation engine with the relevant data available to you as a data scientist. Some things to bear in mind about these types of models:

3. Things to bear in mind

If users don't have a lot of ratings, and ALS can't infer much about them, it's likely that ALS will make broad general recommendations that aren't really personalized. You might have seen this if you spent extra time exploring some of the recommendation output. Like all models, the more data there is, the better the model performs.

4. Things to bear in mind (cont.)

While we've gone over different ways of evaluating recommendations engines, the only way to really know if your model performs well is to test it on users and see if they actually take your recommendations. It's entirely possible that a simple binary implicit ratings model provides better recommendations for users than an explicit model, but the only way to know is to test it. Bear this in mind as you move forward.

5. Resources

Here are some resources to help you as you continue to learn about these models and begin to build them on your own. The {{1}} first is a paper published by McKinsey and Company discussing the power of recommendation engines like ALS based models. The {[2}} second is the code to build the wide_to_long function discussed in the section about preparing data for ALS. The {[3}} third is the white paper that provides the academic background for building ALS models using implicit ratings. I highly recommend reading this paper as it provides a lot of context and insight into how these models work and alternative ways to evaluate them. The {{4}} fourth is a GitHub link for code that manages the cross validation and model evaluation for implicit ratings models using ALS in Pyspark. The {[5}} last resource listed here is a paper that discusses the math and intuition behind the user-based weighting and item-based weighting methodologies for addressing the class imbalance present in binary ratings models. Congratulations on completing this course, and best of luck as you move forward.